In [1]:
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import tensorflow as tf
import os
import glob
import skimage.io
import joblib
from PIL import Image
import itertools
from sklearn.model_selection import train_test_split

In [2]:
from labelMaker import labelMaker

In [59]:
arr = np.ones((10, 5, 5, 200, 200, 3))

In [60]:
arr[0, 0, 0, :, :, :].shape

(200, 200, 3)

### Build dataset from data team's format

In [3]:
def build_dataset_from_arr(data_file, puzzle_dims=(5,5)):
    
    im_list = []
    #y_list = []
    #X_list = []
    
    
    
    data_arr = joblib.load(data_file)
    
    X_list = np.empty((2148000, data_arr.shape[3], data_arr.shape[4], 6))
    y_list = np.empty((2148000, 5))
    
    pieceIndexs = [0, 1, 2, 3, 4]
    
    coordinates = []
    for i in pieceIndexs:
        for j in pieceIndexs:
            coordinates.append((i, j))
    #print("COORDS", coordinates)
    pairsList = list(itertools.permutations(coordinates, 2))
#     for tupleIndex in range(len(pairsList)):
#         originalTuple = pairsList[tupleIndex]
#         newOne = originalTuple[0] - 1
#         newTwo = originalTuple[1] - 1
#         pairsList[tupleIndex] = (newOne, newTwo)
    #print("COORDS", coordinates)
    #print(len(pairsList))
    #print(pairsList)
    counter = 0
    for puzz_idx in range(data_arr.shape[0]):
        
        
        for pair in pairsList:
            tup1 = pair[0]
            tup2 = pair[1]
        
#         for i in range(data_arr.shape[1]):
#             for j in range(data_arr.shape[2]):
                    
#                 im_list.append((data_arr[puzz_idx, i, j, :, :, :], (i,j)))
#         for tup in pairsList:


            X_list[counter] = (
                np.reshape(
                    np.concatenate(
                        (data_arr[puzz_idx][tup1[0]][tup1[1]], data_arr[puzz_idx][tup2[0]][tup2[1]])
                    ),
                    (data_arr.shape[3], data_arr.shape[4], 6)
                )
            )
            y_list[counter] = (np.array(labelMaker(tup1[0],tup1[1],tup2[0],tup2[1])))

            counter += 1

            



#         print("phase 1")
#         counter = 0
#         for i in range(len(im_list)):
#             for j in range(len(im_list)):
                
#                 if not i == j:
#                     X_list[counter] = (
#                         np.reshape(
#                             np.concatenate(
#                                 (im_list[i][0], im_list[j][0])
#                             ),
#                             (data_arr.shape[3], data_arr.shape[4], 6)
#                         )
#                     )
#                     y_list[counter] = (np.array(labelMaker(im_list[i][1][0],im_list[i][1][1],im_list[j][1][0],im_list[j][1][1])))

#                     counter += 1
    
#     full_X = np.empty((len(X_list), *X_list[0].shape))
#     print('DEBUG: '+ str(full_X.shape[0]) + ' ' + str(full_X.shape[1]) + ' ' + str(full_X.shape[2]) + ' ' + str(full_X.shape[3]))
#     full_y = np.empty((len(y_list), 5))
    
#     for i in range(len(X_list)):
#         #print(X_list[i])
#         full_X[i] = X_list[i]
#         #print(y_list[i].tolist())
#         full_y[i] = y_list[i].tolist()
        
    
    return X_list, y_list

In [4]:
data_path = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\allImages_20'

In [5]:
X, y = build_dataset_from_arr(data_path)

In [25]:
sample_X, sample_y = X[:100], y[:100]

In [6]:
X.shape

(2148000, 20, 20, 6)

In [7]:
def preprocess_norm(X):
    X = X / 255.0
    
    return X

In [8]:
def equalize_labels(X, y, pct_connected=0.5):
    print(y)
    print(np.where(y[:,4] == 0))
    adj_X = X[np.where(y[:,4] == 0)]
    adj_y = y[np.where(y[:,4] == 0)]
    
    print(adj_X.shape)
    
    n_adj_X = X[np.where(y[:,4] == 1)]
    n_adj_y = y[np.where(y[:,4] == 1)]
    
    total_adj = adj_X.shape[0]

    idx_mask = np.random.choice(np.arange(n_adj_X.shape[0]), int((total_adj/pct_connected)-total_adj) , replace=False)
    
    X = np.concatenate((n_adj_X[idx_mask], adj_X))
    y = np.concatenate((n_adj_y[idx_mask], adj_y))
    
    return X, y
    

In [9]:
X = preprocess_norm(X)

In [10]:
X, y = equalize_labels(X, y)

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]
(array([      0,       4,      24, ..., 2147975, 2147995, 2147999],
      dtype=int64),)
(286400, 20, 20, 6)


In [11]:
X.shape

(572800, 20, 20, 6)

In [12]:
y.shape

(572800, 5)

In [13]:
save_path_X = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\preprocessedData_X_20_50split'
save_path_y = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\preprocessedData_y_20_50split'

In [14]:
joblib.dump(X, save_path_X, compress=0)
joblib.dump(y, save_path_y, compress=0)

['C:\\machine learning workspace\\final project\\MachineLearningTeamProject\\Model\\preprocessedData_y_20_50split']

### Simple Model Playground

In [132]:
y

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]])

In [2]:
load_path_X = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\preprocessedData_X_20_50split'
load_path_y = r'C:\machine learning workspace\final project\MachineLearningTeamProject\Model\preprocessedData_y_20_50split'

In [3]:
X = joblib.load(load_path_X)
y = joblib.load(load_path_y)

In [7]:
X.shape

(572800, 20, 20, 6)

In [21]:
y.shape

(572800, 5)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
def build_CNN_w_pooling(X):    
    model = Sequential()

    model.add(Conv2D(64,(3,3),input_shape=X.shape[1:] ,activation = 'relu',padding='same',))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.2))
    
    model.add(Conv2D(64,(3,3),input_shape=X.shape[1:] ,activation = 'relu',padding='same',))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    #model.add(Dropout(0.2))
    
    model.add(Conv2D(64,(3,3),input_shape=X.shape[1:] ,activation = 'relu',padding='same',))
    #model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    #model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())

    
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
   
    
    model.add(Dense(5, activation='softmax'))

    model.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [7]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [6]:
model = build_CNN_w_pooling(X)
model.fit(X_train, y_train , epochs=150, batch_size=4096, validation_split=0.1)

Epoch 1/150
101/101 [==============================] - 36s 263ms/step - loss: 1.4352 - accuracy: 0.4914 - val_loss: 1.3553 - val_accuracy: 0.5015
Epoch 2/150
101/101 [==============================] - 24s 240ms/step - loss: 1.3223 - accuracy: 0.5152 - val_loss: 1.2268 - val_accuracy: 0.5614
Epoch 3/150
101/101 [==============================] - 24s 241ms/step - loss: 1.1921 - accuracy: 0.5734 - val_loss: 1.1084 - val_accuracy: 0.5867
Epoch 4/150
101/101 [==============================] - 24s 241ms/step - loss: 1.1230 - accuracy: 0.5883 - val_loss: 1.0654 - val_accuracy: 0.5952
Epoch 5/150
101/101 [==============================] - 24s 242ms/step - loss: 1.1016 - accuracy: 0.5923 - val_loss: 1.0508 - val_accuracy: 0.5960
Epoch 6/150
101/101 [==============================] - 24s 242ms/step - loss: 1.0724 - accuracy: 0.5956 - val_loss: 1.0189 - val_accuracy: 0.6005
Epoch 7/150
101/101 [==============================] - 24s 242ms/step - loss: 1.0562 - accuracy: 0.5967 - val_loss: 1.0269 -

Epoch 57/150
101/101 [==============================] - 24s 241ms/step - loss: 0.6478 - accuracy: 0.7129 - val_loss: 0.5465 - val_accuracy: 0.7828
Epoch 58/150
101/101 [==============================] - 24s 241ms/step - loss: 0.6513 - accuracy: 0.7118 - val_loss: 0.5509 - val_accuracy: 0.7770
Epoch 59/150
101/101 [==============================] - 24s 241ms/step - loss: 0.6400 - accuracy: 0.7174 - val_loss: 0.5487 - val_accuracy: 0.7801
Epoch 60/150
101/101 [==============================] - 24s 241ms/step - loss: 0.6305 - accuracy: 0.7232 - val_loss: 0.5436 - val_accuracy: 0.7822
Epoch 61/150
101/101 [==============================] - 24s 241ms/step - loss: 0.6372 - accuracy: 0.7200 - val_loss: 0.5339 - val_accuracy: 0.7889
Epoch 62/150
101/101 [==============================] - 24s 242ms/step - loss: 0.6318 - accuracy: 0.7223 - val_loss: 0.5458 - val_accuracy: 0.7834
Epoch 63/150
101/101 [==============================] - 24s 241ms/step - loss: 0.6301 - accuracy: 0.7235 - val_loss: 0

101/101 [==============================] - 24s 241ms/step - loss: 0.5507 - accuracy: 0.7693 - val_loss: 0.4981 - val_accuracy: 0.8081
Epoch 113/150
101/101 [==============================] - 24s 241ms/step - loss: 0.5537 - accuracy: 0.7665 - val_loss: 0.4802 - val_accuracy: 0.8201
Epoch 114/150
101/101 [==============================] - 24s 241ms/step - loss: 0.5485 - accuracy: 0.7704 - val_loss: 0.4802 - val_accuracy: 0.8190
Epoch 115/150
101/101 [==============================] - 24s 242ms/step - loss: 0.5510 - accuracy: 0.7673 - val_loss: 0.4765 - val_accuracy: 0.8200
Epoch 116/150
101/101 [==============================] - 24s 241ms/step - loss: 0.5521 - accuracy: 0.7680 - val_loss: 0.4838 - val_accuracy: 0.8137
Epoch 117/150
101/101 [==============================] - 24s 241ms/step - loss: 0.5467 - accuracy: 0.7710 - val_loss: 0.4836 - val_accuracy: 0.8170
Epoch 118/150
101/101 [==============================] - 24s 241ms/step - loss: 0.5487 - accuracy: 0.7702 - val_loss: 0.4822 -

In [8]:
model_save_path = r"C:\machine learning workspace\final project\MachineLearningTeamProject\Model\savedModel81"
model.save(model_save_path)

INFO:tensorflow:Assets written to: C:\machine learning workspace\final project\MachineLearningTeamProject\Model\savedModel81\assets


In [7]:
loss, accuracy = model.evaluate(X_test, y_test)

3580/3580 [==============================] - 8s 2ms/step - loss: 0.4807 - accuracy: 0.8203


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 20, 64)        3520      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 dropout (Dropout)           (None, 20, 20, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 dropout_1 (Dropout)         (None, 20, 20, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 20, 20, 128)       1

In [12]:
loadedModel = keras.models.load_model(model_save_path)

In [13]:
loadedModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 20, 64)        3520      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 64)        36928     
                                                                 
 dropout (Dropout)           (None, 20, 20, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 dropout_1 (Dropout)         (None, 20, 20, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 20, 20, 128)       1

In [113]:
model.predict(X)

array([[0.12516342, 0.10846931, 0.11553416, 0.10564344, 0.5451897 ],
       [0.09151299, 0.07960231, 0.07611708, 0.07968806, 0.67307955],
       [0.08452865, 0.06726538, 0.05977225, 0.06250986, 0.7259239 ],
       ...,
       [0.10909431, 0.10055214, 0.1113238 , 0.11079023, 0.5682395 ],
       [0.09062684, 0.08424608, 0.08398275, 0.09016786, 0.6509765 ],
       [0.10043259, 0.09283187, 0.09430876, 0.10005014, 0.6123767 ]],
      dtype=float32)

In [114]:
import matplotlib.pyplot as plt

In [120]:
X[0,:,:,:3].shape

(200, 200, 3)

In [142]:
X[0,:,:,:3]

array([[[ 96.,  89.,  63.],
        [121., 114.,  88.],
        [139., 132., 104.],
        ...,
        [185.,  66.,  36.],
        [178.,  65.,  36.],
        [171.,  63.,  31.]],

       [[ 99.,  92.,  66.],
        [115., 107.,  81.],
        [132., 123.,  96.],
        ...,
        [186.,  66.,  32.],
        [179.,  66.,  32.],
        [173.,  62.,  28.]],

       [[112., 102.,  77.],
        [116., 104.,  78.],
        [127., 116.,  88.],
        ...,
        [186.,  64.,  27.],
        [182.,  65.,  28.],
        [177.,  64.,  26.]],

       ...,

       [[ 40.,  47.,  39.],
        [ 40.,  47.,  39.],
        [ 38.,  45.,  37.],
        ...,
        [ 58.,  61.,  54.],
        [ 59.,  62.,  55.],
        [ 59.,  61.,  56.]],

       [[ 42.,  49.,  41.],
        [ 43.,  50.,  42.],
        [ 40.,  47.,  39.],
        ...,
        [ 51.,  52.,  46.],
        [ 53.,  56.,  49.],
        [ 54.,  56.,  51.]],

       [[ 42.,  49.,  41.],
        [ 42.,  49.,  41.],
        [ 40.,  

In [10]:
plt.imshow(X[0,:,:,3:])

NameError: name 'plt' is not defined